In [2]:
## Contents of https://github.com/wellsoliver/py-gameday/blob/master/lib/__init__.py
## Only one small edit to BASE

from logging import getLogger, Handler
from urllib import urlopen
from warnings import simplefilter
from time import sleep

class NullHandler(Handler):
    def emit(self, record):
        pass

class CONSTANTS:

#     BASE = 'http://gd2.mlb.com/components/game/%LEAGUE%/'
    BASE = 'http://gd2.mlb.com/components/game/mlb/'
    FETCH_TRIES = 10

class Fetcher:
    @classmethod
    def fetch(self, url):
        for i in xrange(CONSTANTS.FETCH_TRIES):
            logger.debug('FETCH %s' % url)
            try:
                page = urlopen(url)
            except IOError, e:
                if i == CONSTANTS.FETCH_TRIES-1:
                    logger.error('ERROR %s (max tries %s exhausted)' % (url, CONSTANTS.FETCH_TRIES))
                sleep(1)
                continue

            if page.getcode() == 404:
                return None
            else:
                return page.read()
            break

logger = getLogger('gameday')
logger.addHandler(NullHandler())

In [3]:
# Edited contents of https://github.com/wellsoliver/py-gameday/blob/master/lib/atbats.py
# Only changed the save function and loaded pandas

from xml.dom import minidom
from bs4 import BeautifulSoup
from re import search
import pandas as pd # JKG Added this

class Pitch:
    def __init__(self, element, count, **kwargs):

        values = {}
        values['num'] = kwargs['num'] if 'num' in kwargs else None
        values['game_id'] = kwargs['game_id'] if 'game_id' in kwargs else None
        values['pitcher'] = kwargs['pitcher'] if 'pitcher' in kwargs else None
        values['batter'] = kwargs['batter'] if 'batter' in kwargs else None
        values['b'] = count['balls']
        values['s'] = count['strikes']
        
        # these change a lot :(
        # tired of taking them from the XML element
        # because maybe I don't have them in the schema
        FIELDS = ['des','id','type','x','y','on_1b','on_2b','on_3b','sv_id','start_speed',
            'end_speed','sz_top','sz_bot','pfx_x','pfx_z','px','pz','x0','y0','z0','vx0','vy0','vz0',
            'ax','ay','az','break_y','break_angle','break_length','pitch_type','type_confidence',
            'spin_dir','spin_rate','zone']

        for key in element.attributes.keys():
            if key in FIELDS:
                values[key] = element.attributes[key].value
        
        self.values = values

class AtBats(list):
    
    def save(self):

## This is the old save function that I overwrote
        
#         DB = store.Store()
#         for inning in self:
#             for atbat in inning:
#                 keys = [k for k in atbat.keys() if k != 'pitches']
#                 values = [None if atbat[k] == '' else atbat[k] for k in keys]
                
#                 print ','.join(keys)

#                 sql ='REPLACE INTO atbat (%s) VALUES(%s)' % (','.join(keys), ','.join(['%s'] * len(keys)))
#                 DB.query(sql, values)
#                 DB.save()

#                 for pitch in atbat['pitches']:
#                     pitch.save()

        AB_FIELDS = ['inning','num','p_throws','event','score','away_team_runs','start_tfs_zulu','des_es','start_tfs',
                  'pitcher','batter','half','game_id','b','b_height','home_team_runs','des','o','s','stand','event2']

        P_FIELDS = ['des','id','type','x','y','on_1b','on_2b','on_3b','sv_id','start_speed',
            'end_speed','sz_top','sz_bot','pfx_x','pfx_z','px','pz','x0','y0','z0','vx0','vy0','vz0',
            'ax','ay','az','break_y','break_angle','break_length','pitch_type','type_confidence',
            'spin_dir','spin_rate','zone']
        
        ab_values = dict((k,[]) for k in AB_FIELDS)
        p_values = dict((k,[]) for k in P_FIELDS)
                
        for inning in self:
            for atbat in inning:
                for key in ab_values.keys():
                    ab_values[key].append(atbat.get(key,None))
                
                for pitch in atbat['pitches']:
                    for key in p_values.keys():
                        p_values[key].append(pitch.values.get(key,None))

        return (pd.DataFrame(ab_values), pd.DataFrame(p_values))

        
    def __init__(self, gid, game_id):
        super(AtBats,self).__init__()

        year, month, day = gid.split('_')[1:4]
        url = '%syear_%s/month_%s/day_%s/%s/inning/' % (CONSTANTS.BASE, year, month, day, gid)        
        
        contents = Fetcher.fetch(url)
        if contents is None:
            return
        
        soup = BeautifulSoup(contents)

        inning_num = 1
        for inning_link in soup.findAll('a'):
            if search(r'inning_\d+\.xml', inning_link['href']):
                inning_url = '%s%s' % (url, inning_link['href'])
                doc = minidom.parseString(Fetcher.fetch(inning_url))
                
                inning = []
                
                for atbat in doc.getElementsByTagName('atbat'):
                    values = {}
                    half = atbat.parentNode.nodeName
                    for key in atbat.attributes.keys():
                        values[str(key)] = atbat.attributes[key].value

                    values['half'] = half
                    values['game_id'] = game_id
                    values['inning'] = inning_num
                    values['pitches'] = []
                    
                    balls = 0
                    strikes = 0
                    for pitch in atbat.getElementsByTagName('pitch'):
                        count = {'balls': balls, 'strikes': strikes}
                        kwargs = {'game_id': game_id,
                            'batter': values['batter'],
                            'pitcher': values['pitcher'],
                            'num': atbat.attributes['num'].value}
                        p = Pitch(pitch, count, **kwargs)
                        values['pitches'].append(p)

                        if pitch.attributes['type'].value == 'B':
                            balls = balls + 1
                        elif pitch.attributes['type'].value == 'S':
                            strikes = strikes + 1

                    inning.append(values)
                self.append(inning)
                inning_num += 1

In [7]:
# import timeit
# start = timeit.default_timer()

(atbats, pitches) = AtBats("gid_2013_04_01_detmlb_minmlb_1", "Game_id").save()

# stop = timeit.default_timer()
# print stop - start 

atbats

,away_team_runs,b,b_height,batter,des,des_es,event,event2,game_id,half,...,inning,num,o,p_throws,pitcher,s,score,stand,start_tfs,start_tfs_zulu
0,0,3,6-1,457706,Austin Jackson singles on a line drive to righ...,Austin Jackson pega sencillo con línea a jardi...,Single,None,Game_id,top,...,1,1,0,R,474699,2,None,R,201646,2013-04-01T20:16:46Z
1,0,1,6-2,116338,Torii Hunter singles on a ground ball to right...,Torii Hunter pega sencillo con rodado a jardin...,Single,None,Game_id,top,...,1,2,0,R,474699,1,None,R,201918,2013-04-01T20:19:18Z
2,1,1,6-4,408234,"Miguel Cabrera grounds into a force out, short...",Miguel Cabrera batea rodado batea para out for...,Forceout,None,Game_id,top,...,1,3,1,R,474699,2,T,R,202108,2013-04-01T20:21:08Z
3,2,1,5-11,425902,Prince Fielder doubles (1) on a ground ball to...,Prince Fielder pega doble (1) con rodado a jar...,Double,None,Game_id,top,...,1,4,1,R,474699,0,T,L,202335,2013-04-01T20:23:35Z
4,2,1,6-2,400121,Victor Martinez called out on strikes.,Victor Martinez se poncha sin tirarle.,Strikeout,None,Game_id,top,...,1,5,2,R,474699,3,None,L,202450,2013-04-01T20:24:50Z
5,2,1,6-0,543108,"Andy Dirks grounds out, second baseman Brian D...","Andy Dirks batea rodado de out, segunda base B...",Groundout,None,Game_id,top,...,1,6,3,R,474699,2,None,L,202650,2013-04-01T20:26:50Z
6,2,1,6-2,543305,Aaron Hicks strikes out swinging.,Aaron Hicks se poncha tirándole.,Strikeout,None,Game_id,bottom,...,1,7,1,R,434378,3,None,L,203118,2013-04-01T20:31:18Z
7,2,0,6-5,408045,Joe Mauer singles on a soft ground ball to thi...,Joe Mauer pega sencillo con rodado suave a ter...,Single,None,Game_id,bottom,...,1,8,1,R,434378,1,None,L,203310,2013-04-01T20:33:10Z
8,2,2,6-2,425545,Josh Willingham pops out to third baseman Migu...,Josh Willingham batea elevadito de out a terce...,Pop Out,None,Game_id,bottom,...,1,9,2,R,434378,1,None,R,203435,2013-04-01T20:34:35Z
9,2,3,6-4,408047,Justin Morneau lines out to right fielder Tori...,Justin Morneau batea línea de out a jardinero ...,Lineout,None,Game_id,bottom,...,1,10,3,R,434378,1,None,L,203632,2013-04-01T20:36:32Z


In [8]:
pitches

,ax,ay,az,break_angle,break_length,break_y,des,end_speed,id,on_1b,...,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,zone
0,-1.019,25.733,-22.099,-1.3,5.6,23.8,Called Strike,81.2,3,None,...,2.000,8.113,-128.299,-4.119,78.11,-2.531,132.97,50.0,6.383,3
1,-2.196,24.895,-19.83,2.3,4.6,23.9,Ball,84.2,4,None,...,2.000,7.721,-132.224,-3.877,94.42,-2.586,125.20,50.0,6.423,12
2,6.517,21.359,-33.67,-12.3,9.6,23.9,Ball,78.0,5,None,...,2.000,7.732,-120.927,-6.583,48.93,-2.291,185.64,50.0,6.409,14
3,-1.775,26.549,-18.883,2.4,4.5,23.8,Called Strike,83.6,6,None,...,2.000,4.93,-132.145,-4.8,121.03,-2.514,130.38,50.0,6.417,11
4,-6.553,27.479,-18.514,16.0,5.1,23.8,Ball,79.8,7,None,...,2.000,4.473,-127.346,-5.033,136.48,-2.479,136.43,50.0,6.405,11
5,-1.5,25.136,-18.557,.8,4.5,23.8,"In play, no out",82.9,8,None,...,2.000,6.755,-130.529,-4.38,97.00,-2.502,126.06,50.0,6.455,11
6,-10.99,25.536,-19.563,24.8,5.7,23.8,Foul,79.4,12,457706,...,2.000,7.487,-125.94,-5.358,109.01,-2.327,139.88,50.0,6.458,5
7,-14.437,30.574,-23.324,27.5,6.7,23.8,Ball,82.2,13,457706,...,2.000,6.187,-131.125,-8.114,134.76,-2.404,172.69,50.0,6.3,13
8,-8.438,24.358,-20.714,18.8,5.4,23.9,"In play, no out",82.1,14,457706,...,2.000,6.065,-129.165,-4.83,120.17,-2.286,134.70,50.0,6.47,1
9,6.16,23.08,-35.521,-10.9,9.7,23.8,Ball,77.1,20,116338,...,2.000,3.892,-121.652,0.514,108.15,-2.359,113.11,50.0,6.711,11
